### SQL and Databases

Structured Query Language or SQL is a very common tool for data science.  It is a way to access data from databases.  We use databases rather than spreadsheets or csv's since they
tend to be more efficient uses of space.

For DTSC 2302 we are going to use a simple version of SQL and of database structure.  Generally, a database would be stored on a server and communications would go through the server.  In this course, we are just going to use some simple databases that are stored locally using a python library called _sqlite3_.  You may have to _install_ this
library but it now comes as part of my python downloads.

In [1]:
# Import the sqlite3 library
import sqlite3 as db
import pandas as pd

The first thing we have to do is open a connection to our database which we do through the command _db.connect(dbname)_ where *dbname* is the name of a database.

We then also have to define a _cursor_ so that we can interact with the connection.  Basically the cursor acts like the connection to a database server in a usaal SQL setting 
where you have a local machine and the database is on the server.

In [2]:
# create a connection between the database and VSCode
# If the database does not exist, SQLite will create it.
conn = db.connect('students_courses.db')  # Create or connect to database
# create a cursor object which will be the way through which we pass commands
cursor = conn.cursor()

Having created the database --- which should appear in your current directory --- we will next create a table.  We use the cursor to send a command that 
we want to execute to the database.  We'll do a simple table called _student_ here with some student information.

When we CREATE a TABLE, we need to specify the attributes as well as the PRIMARY KEY.  For each attribute we also specify the data type.  The most common
data types are: INTEGER, REAL, and TEXT.  The REAL datatype is like FLOAT in some other programming languages.

In [3]:
#Create a TABLE called students which will have fields: student_id, first_name, last_name, age and course_id

cursor.execute('''
CREATE TABLE IF NOT EXISTS student (
    student_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    age INTEGER,
    course_id INTEGER 
)
''')

To enter data by hand into a database we create an object, as below, which I've called *students_data*.  We will then INSERT that data into our table called *student*.

In [4]:

# Step 4: Insert some data into the tables
# Insert data into the 'students' table
students_data = [
    (165, 'John', 'Doe', 20, 2),
    (245, 'Jane', 'Smith', 22, 2),
    (3333, 'Mike', 'Jordan', 21, 1),
]

In the next chunk/cell of code we INSERT the *students_data* into the table called *student*.

In [5]:


cursor.executemany('''
INSERT INTO student (student_id, first_name, last_name, age, course_id)
VALUES (?, ?, ?, ?, ?)
''', students_data)

Suppose we want to look at the values in our table.  The next cell of code shows you how to do that.

The first line 'SELECT * FROM student' says to SQL from the table *student* give me all (denoted by the *) the attributes and all the rows.

The rest of the commands in that cell get the output from cursor and then print it.

In [6]:
cursor.execute('SELECT * FROM student')
rows=cursor.fetchall()
for row in rows:
    print(row)

(165, 'John', 'Doe', 20, 2)
(245, 'Jane', 'Smith', 22, 2)
(3333, 'Mike', 'Jordan', 21, 1)


We don't have to get all the attributes from a table when we SELECT we can just get some of them.  In the cell below, the first line says just give
me the *first_name* and *last_name* from the table *student*.  Then print those.

In [7]:
cursor.execute('SELECT first_name, last_name FROM student')
rows=cursor.fetchall()
for row in rows:
    print(row)

('John', 'Doe')
('Jane', 'Smith')
('Mike', 'Jordan')


We also don't have to get all of the instances/rows from a table but rather we get just the first _n_ where _n_ has to be an integer.  To do that
we replace the *fetchall* above with *fetchmany* and we 

In [8]:
cursor.execute('SELECT age,first_name FROM student')
rows=cursor.fetchmany(2)
for row in rows:
    print(row)

(20, 'John')
(22, 'Jane')


Next we will create a table called *course* with information about the course.

In [9]:

# Courses table: Contains information about courses.
cursor.execute('''
CREATE TABLE IF NOT EXISTS course (
    course_id INTEGER PRIMARY KEY,
    course_name TEXT,
    instructor TEXT
)
''')

Next we create an array of values to go into the table *course*.

In [10]:

# Insert data into the 'courses' table
courses_data = [
    (1, 'Python 101', 'Dr. Alice'),
    (2, 'Data Science', 'Prof. Bob'),
    (3, 'Machine Learning', 'Dr. Charlie'),
]

We insert those values into *course* with the following code.

In [11]:
#Insert data into the 
cursor.executemany('''
INSERT INTO course (course_id, course_name, instructor)
VALUES (?, ?, ?)
''', courses_data)

Often we want to combine the information in one or more tables.  The usual *verb* for doing such a thing in data science is a 'JOIN', so we want
to join information from one table to another.  Below in the code we SELECT attributes *table1.attributeX1*, *table2.attributeX2*,..., *table2.attributeY1*
FROM *table1* JOIN *table2* using keys.  The keys we use below are *course_id* from both tables.  The keys we use may not be have the same names for their
attributes so we use the *table.attribute* structure to specify the keys.

In [12]:

# Let's join the two tables (students and courses) based on student ID and course ID
cursor.execute('''
SELECT student.first_name, student.last_name, student.age, course.course_name, course.instructor
FROM student
JOIN course ON student.course_id = course.course_id
''')

The cell below prints the output from our JOIN above.  As you can see once you run this, the values from the two tables have been combined.

In [13]:
# Fetch all the rows of the result
rows = cursor.fetchall()


# Display the results
for row in rows:
    print(row)

('John', 'Doe', 20, 'Data Science', 'Prof. Bob')
('Jane', 'Smith', 22, 'Data Science', 'Prof. Bob')
('Mike', 'Jordan', 21, 'Python 101', 'Dr. Alice')


Once you are done with a database you should close the connection.  We will revisit JOINs again soon.

In [14]:
conn.close()

Next we are going to look at reading in data from a CSV file directly into a database.  Luckily there is a nice function that lets us move a pandas dataframe to an SQL table.
That command is *to_sql* and it takes the following arguments *table_name*, *cursor_name* and *if_exists*.  The former two give the name of the table to put the dataframe
values into and name of the cursor mechanism for the particular sqlite3 database.  The last argument, *if_exists* has three options: 'fail', 'replace','append'.  This argument tells
python what to do if the table already exists in the database.  The default here is 'fail' since you don't want to mistakenly overwrite a table in a database. 

In [15]:
df=pd.read_csv('https://webpages.charlotte.edu/mschuck1/classes/DTSC2301/Data/Charlotte_football_2024_games_small.csv')


connection = db.connect('football.db')

cursor = connection.cursor()

df.to_sql('game',connection,if_exists='replace')

12

Print the values in the table *game*.

In [16]:
cursor.execute('SELECT * FROM game')
rows=cursor.fetchall()
for row in rows:
    print(row)


(0, 401643774, 2024, 1, 'regular', '2024-09-01T00:00:00.000Z', 'Jerry Richardson Stadium', 2429, 'Charlotte', 'American Athletic', 7, 256, 'James Madison', 'Sun Belt', 30)
(1, 401635537, 2024, 2, 'regular', '2024-09-07T19:30:00.000Z', 'Kenan Stadium', 153, 'North Carolina', 'ACC', 38, 2429, 'Charlotte', 'American Athletic', 20)
(2, 401645327, 2024, 3, 'regular', '2024-09-14T22:00:00.000Z', 'Jerry Richardson Stadium', 2429, 'Charlotte', 'American Athletic', 27, 2241, 'Gardner-Webb', 'Big South-OVC', 26)
(3, 401628486, 2024, 4, 'regular', '2024-09-21T16:00:00.000Z', 'Memorial Stadium (Bloomington, IN)', 84, 'Indiana', 'Big Ten', 52, 2429, 'Charlotte', 'American Athletic', 14)
(4, 401645333, 2024, 5, 'regular', '2024-09-28T23:00:00.000Z', 'Rice Stadium', 242, 'Rice', 'American Athletic', 20, 2429, 'Charlotte', 'American Athletic', 21)
(5, 401645340, 2024, 6, 'regular', '2024-10-05T19:30:00.000Z', 'Jerry Richardson Stadium', 2429, 'Charlotte', 'American Athletic', 55, 151, 'East Carolina',

When you are done with a database you should close the connection.

In [17]:
connection.close()

### Tasks


1. We are going to create a database called bank.db and in that database create two tables.  The first table is called accounts and has attributes: *account_id*, *member_name* and *date_opened*.  The second table is called transactions and has attributes: *transaction_id*, *account_id*, *date*, *amount*.  Before you create that account, think about the variable types of each attribute.


In [19]:
# create a database called bank.db
conn = db.connect('bank.db')
# in bank.db create two tables called accounts and transactions

cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS accounts (
    account_id INTEGER PRIMARY KEY,
    member_name TEXT,
    date_opened REAL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS transactions (
    transaction_id INTEGER PRIMARY KEY,
    account_id INTEGER,
    date REAL,
    amount REAL
)
''')


2. Create the database from Task 1 and enter three instances of accounts and two instances of transactions.


In [25]:
# enter 3 instances of data into the accounts table
accounts_data = [
    (4, 'John Doe', '2022-01-01'),
    (5, 'Jane Doe', '2022-02-01'),
    (6, 'Bob Smith', '2022-03-01')
]

cursor.executemany('''
INSERT INTO accounts (account_id, member_name, date_opened)
VALUES (?, ?, ?)
''', accounts_data)

# enter 2 instances of data into the transactions table
transactions_data = [
    (1, 4, '2022-01-02', 100.0),
    (2, 5, '2022-02-02', 200.0)
]

cursor.executemany('''
INSERT INTO transactions (transaction_id, account_id, date, amount)
VALUES (?, ?, ?, ?)
''', transactions_data)


3. Reconnect to the *football.db* database and print the first 5 instances.


In [27]:
connection = db.connect('football.db')

cursor = connection.cursor()

df.to_sql('game',connection,if_exists='replace')

# print first 5 instances of the game table
cursor.execute('SELECT * FROM game')
rows=cursor.fetchmany(5)
for row in rows:
    print(row)

(0, 401643774, 2024, 1, 'regular', '2024-09-01T00:00:00.000Z', 'Jerry Richardson Stadium', 2429, 'Charlotte', 'American Athletic', 7, 256, 'James Madison', 'Sun Belt', 30)
(1, 401635537, 2024, 2, 'regular', '2024-09-07T19:30:00.000Z', 'Kenan Stadium', 153, 'North Carolina', 'ACC', 38, 2429, 'Charlotte', 'American Athletic', 20)
(2, 401645327, 2024, 3, 'regular', '2024-09-14T22:00:00.000Z', 'Jerry Richardson Stadium', 2429, 'Charlotte', 'American Athletic', 27, 2241, 'Gardner-Webb', 'Big South-OVC', 26)
(3, 401628486, 2024, 4, 'regular', '2024-09-21T16:00:00.000Z', 'Memorial Stadium (Bloomington, IN)', 84, 'Indiana', 'Big Ten', 52, 2429, 'Charlotte', 'American Athletic', 14)
(4, 401645333, 2024, 5, 'regular', '2024-09-28T23:00:00.000Z', 'Rice Stadium', 242, 'Rice', 'American Athletic', 20, 2429, 'Charlotte', 'American Athletic', 21)



4. Having reconnected to the *football.db* database, print the scores for all of the games.  

In [29]:
# print scores of all the games
cursor.execute('SELECT * FROM game')
rows=cursor.fetchall()
for row in rows:
    print(row)

(0, 401643774, 2024, 1, 'regular', '2024-09-01T00:00:00.000Z', 'Jerry Richardson Stadium', 2429, 'Charlotte', 'American Athletic', 7, 256, 'James Madison', 'Sun Belt', 30)
(1, 401635537, 2024, 2, 'regular', '2024-09-07T19:30:00.000Z', 'Kenan Stadium', 153, 'North Carolina', 'ACC', 38, 2429, 'Charlotte', 'American Athletic', 20)
(2, 401645327, 2024, 3, 'regular', '2024-09-14T22:00:00.000Z', 'Jerry Richardson Stadium', 2429, 'Charlotte', 'American Athletic', 27, 2241, 'Gardner-Webb', 'Big South-OVC', 26)
(3, 401628486, 2024, 4, 'regular', '2024-09-21T16:00:00.000Z', 'Memorial Stadium (Bloomington, IN)', 84, 'Indiana', 'Big Ten', 52, 2429, 'Charlotte', 'American Athletic', 14)
(4, 401645333, 2024, 5, 'regular', '2024-09-28T23:00:00.000Z', 'Rice Stadium', 242, 'Rice', 'American Athletic', 20, 2429, 'Charlotte', 'American Athletic', 21)
(5, 401645340, 2024, 6, 'regular', '2024-10-05T19:30:00.000Z', 'Jerry Richardson Stadium', 2429, 'Charlotte', 'American Athletic', 55, 151, 'East Carolina',